# Web Scraping!!!!

## Morning Objectives

1. Understand motivation for web scraping:
    * What does a web data pipeline look like?
    * How should we store data from the web?
2. Know high level differences between NoSQL and SQL.
3. Perform basic operations using (Py)Mongo.

<div style="text-align: center"><h3>The Reality of Scraping</h3><img src="images/scraping_meme.png" style="width: 600px"></div>

## Why do we scrape the web?

* Realistically, data that you want to study won't always be avaliable to you in the form of a curated data set.
* Need to go to the internets to find interesting data:
    * From an existing company
    * Text for NLP
    * Images
    <div style="text-align: center"><h3>Web Data Pipeline</h3><img src="images/web_data_pipeline.png" style="width: 600px"></div>

## Storing data from the web

* We already know how to store data -> SQL (RBDMS).
    * Why wouldn't SQL necessarily be the best tool for storing data that we retrieve from the web?
        * Data are messy!
* Enter No SQL. Stands for **N**ot **o**nly **SQL**. MongoDB is a flavor of NoSQL, like PosgreSQL is a flavor of SQL.
    * A NoSQL paradigm may be preferable to SQL because it is schemaless.
    * Great for storing unstructured data, as we may find on the web!
    * MongoDB is a document-oriented DBMS:
      <div style="text-align: center"><h3>Centered around "Documents"</h3><img src="images/document_based_storage.png" style="width: 600px"></div>

## SQL vs. Mongo

* SQL - want to prevent reduncancy in data by having tables with unique information and relations between them (normalized data).
    * Creates a framework for querying with joins.
    * Makes it easier to update database. Only ever have to change information in a single place.
    * This can result in "simple" queries being slower, but more complex queries are often faster.
* Mongo - document based storage system. Does not enforce normalized data. Can have data redundancies in documents (denormalized data).
    * No joins.
    * A change to database generally results in needing to change many documents.
    * Since there is redundancy in the documents, simple queries are generally faster. But complex queries are often slower.
    

|         | SQL          | Mongo          |
|---------|--------------|----------------|
| Schema  | Yes => Joins | No => No Joins |
| Storage | Table        | Collection     |
|         | Row          | Document       |
|         | Column       | Field          |

## Connecting to Mongo

In practice, there two main ways that you will be connecting with mongo:

* From Python
* From the console - shell
    
<div style="text-align: center"><img src="images/mongo_clients.png" style="width: 600px"></div>

Both of these clients require a Mongo server to be running. In practice this will require you to start a Mongo Daemon process. To do this exectute the command `mongod` at the terminal.

* Note: The Mongo Daemon will need to occupy the terminal that you started it in for the life of the server session. Read: run `mongod` in a seperate terminal tab (or tmux).

Now we're going to explore the interacting with the mongo shell. This is not the main way that you'll be interacting with mongo while scraping, we have Python for that. But it's good to know how to issue queries from the shell for various reasons.

# Mongo Shell Demo Code

## Using Mongo - General Commands for Inspecting Mongo

```javascript
help                        // List top level mongo commands

db.help()                   // List database level mongo commands

db.<collection name>.help() // List collection level mongo commands.

show dbs                    // Get list of databases on your system

use <database name>         // Change the database that you're current using

show collections            // Get list of collections within the database that you're currently using
```

## Inserting

Once you're using a database you refer to it with the name **db**. Collections within databases are accessible through dot notation.

```javascript
db.users.insert({ name: 'Jon', age: '45', friends: [ 'Henry', 'Ashley']})

db.getCollectionNames()  // Another way to get the names of collections in current database

db.users.insert({ name: 'Ashley', age: '37', friends: [ 'Jon', 'Henry']})
db.users.insert({ name: 'Frank', age: '17', friends: [ 'Billy'], car : 'Civic'})

db.users.find()

    { "_id" : ObjectId("573a39"), "name" : "Jon", "age" : "45", "friends" : [ "Henry", "Ashley" ] }
    { "_id" : ObjectId("573a3a"), "name" : "Ashley", "age" : "37", "friends" : [ "Jon", "Henry" ] }
    { "_id" : ObjectId("573a3b"), "name" : "Frank", "age" : "17", "friends" : [ "Billy" ], "car" : "Civic" }
```

Things to note:
* The three documents that we inserted into the above database didn't all have the same fields.
* Mongo creates an ` _id` field for each document if one isn't provided.

## Querying

```javascript
db.users.find({ name: 'Jon'})                       // find by single field

db.users.find({ car: { $exists : true } })          // find by presence of field

db.users.find({ friends: 'Henry' })                 // find by value in array

db.users.find({}, { name: true })                   // field selection (only return name)
```

A quick way to figure out how to write a Mongo query is to think about how you would do it in SQL and check out a resource like this Mongo endorsed [conversion guide](https://docs.mongodb.com/manual/reference/sql-comparison/#create-and-alter), or use something like a [query translator](http://www.querymongo.com/).

## Updating

```javascript
db.users.update({name: "Jon"}, { $set: {friends: ["Phil"]}})            // replaces friends array

db.users.update({name: "Jon"}, { $push: {friends: "Susie"}})            // adds to friends array

db.users.update({name: "Stevie"}, { $push: {friends: "Nicks"}}, true)   // upsert

db.users.update({}, { $set: { activated : false } }, false, true)       // multiple updates
```

## Imports and Cursors

To import existing data into a mongo database one uses `mongoimport` at the command line. In this way mongo will accept a number of data types: JSON, CSV, and TSV.

```
mongoimport --db tweets --collection coffee --file coffee-tweets.json
```

Now that we have some larger data we can see that returns from queries are not always so small.

```javascript
use tweets
db.coffee.find()
```

When the return from a query will display up to the first 20 documents, after that you will need to type `it` to get more. The cursor that it returns is actually an object that has many methods implemented on it and supports the command `it` to iterate through more return items.

```javascript
db.coffee.find().count()      // 122

db.coffee.find().limit(2)     // Only two documents

db.coffee.find().sort({ 'user.followers_count' : -1}).limit(3)  // Top three users by followers count
```

## Iteration

MongoDB also has a flexible shell/driver. This allows you take some action based on a query or update documents you can use an iterator on the cursor to go document by document. In the Javascript shell we can do this with Javascript's `forEach`. `forEach` is similar to Python's iteration with the `for` loop; however, Javascript actually has a more functional approach to this type of iteration and requires that you pass a callback, a function, similar to `map` and `reduce`.

```javascript
db.coffee.find().forEach(function(doc) {
    doc.entities.urls.forEach(function(url) {
        db.urls.insert({ 'url' : url, 'user': doc.user });
    });
});
```

## Aggregation

Aggregations in Mongo end up being way less pretty than in SQL/Pandas. Let's just bite the bullet and take a look:

```
db.coffee.aggregate( [ { $group :
    {
        _id: "$place.country_code",
        total: { $sum: "$retweet_count" }
    }
}])
```

Here we are first declaring that we're going to do some sort of grouping operation. Then, as Mongo desires everything to have an `_id` field, we specify that the `_id` is going to be the country code that is contained within the place field. And then we're going to perform a sum over the retweet counts within each country code; this information is going to be stored in a field called `total`. What do we get back?

```
db.coffee.find({ retweet_count : { $gt : 0 }  })
```

For a guide on how to convert from an SQL style aggregation to a Mongo style aggreagetion, check out this [aggreagtion conversion guide](https://docs.mongodb.com/manual/reference/sql-aggregation-comparison).

## Afternoon Objectives

1. Understand the process of getting data from the web.
2. Know the basics of HTML/CSS:
    * Know how to pull desired data from web pages.
3. Be able to use existing API's to get fetch pre-formatted data.

### Internet vs. World Wide Web

* The internet is commonly refered to as a network of networks. It is the infrastructure that allow networks all around the world to connect with one another. There are many different protocols to transfer information within this larger, meta-network.
* The World Wide Web, or Web, provides one of the ways that data can be transfered over the internet. Uses a **U**niform **R**esource **L**ocator, URL, to specify the location, within the internet, of a document.

    <div style="text-align: center"><h3>Anatomy of a URL</h3><img src="images/url.png" style="width: 600px"></div>
    
* Documents on the web are generally written in **H**yper**T**ext **M**arkup **L**anguage, HTML, which can be natively viewed by browsers, the tool that we use to browse the web.

### Communication on the Web

Information is transmitted around the web through a number of protocols. The main one that you will see is the **H**yper**T**ext **T**ransfer **P**rotocol, HTTP. These transfers, called **requests**, are initiated in a number of ways, but always begin with the client, read: you at your browser.

 <div style="text-align: center"><h3>Requests in Action</h3><img src="images/requests.png" style="width: 600px"></div>
 
There are 4 main types of request that can be issued by your browser: get, post, put and delete. For web scraping purposes, you will almost always be using get requests. We will learn some more about the others in a couple of weeks during data products day.

# Scraping from a Web Page with Python

Scraping a web site basically comes down to making a request from Python and parsing through the HTML that is returned from each page. For each of these tasks we have a Python library, `requests` and `bs4`, respectively.

### Requests Library

The [requests](http://docs.python-requests.org/en/latest/index.html) library is designed to simplify the process of making http requests within Python. The interface is mindbogglingly simple. Instantiate a requests object to the request, this will mostly be a `get`, with the URL and optional parameters you'd like passed through the request. That instance make the results of the request available via attributes/methods.

In [ ]:
import requests
seven_by_seven = 'http://www.7x7.com'
r = requests.get(seven_by_seven)
r.text

### Getting Info from a Web Page

Now that we can gain easy access to the HMTL for a web page, we need some way to pull the desired content from it. Luckily there is already a system in place to do this. With a combination of HMTL and CSS selectors we can identify the information on a HMTL page that we wish to retrieve and grab it with [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree).

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')
soup.title

In [ ]:
soup.find_all('a')

In [ ]:
soup.select('a')

### Very cool resource for learning about CSS selectors: http://flukeout.github.io/

In [ ]:
for tag in soup.select('h2 a'):
    url = tag['href']
    print url if '7x7' in url else seven_by_seven + url

In [ ]:
total_current_pages = 2054
seven_by_seven_base_url = seven_by_seven + '/?page=0%2C{}'
for page in xrange(0, total_current_pages + 1):
    page_r = requests.get(seven_by_seven_base_url.format(page))
    soup = BeautifulSoup(page_r.text, 'html.parser')

As you go through a web site you should build up a dictionary for the documents that you want to store in Mongo. IN the example above we may, for each post url, create a dictionary with the information:
```python
    { url: url_of_post,
      date: date_of_post,
      poster: name_of poster,
      content: text_from_post }
```

We can then insert these dictionaries into a Mongo database via PyMongo, which we will learn about next.

# Scraping from an Existing API

Let's take a look at the API for all the publically avaliable policing data in the [UK](https://data.police.uk/docs/). After taking a look at the documentation for the interface, let's experiment with what we get when we issue a request to this API. The process looks remarkable similar to the one we went through for scraping a web page, except this time the response we're looking for is avaliable via the `json()` method.

In [ ]:
r = requests.get('https://data.police.uk/api/crimes-street-dates')
r.json()

## API Scraping and Mongo

Many APIs will give you a choice of how it will return data to you, choosing json will make life easier since we will frequently be using Mongo for our storage unit during our scraping endeavors, and it plays very well with json. 

Interacting with Mongo from Python is done with the other Mongo client that we talked about earlier PyMongo. It is designed to have a similar interface as the Mongo shell does, this ends up being fairly intuitive since both Python and JavaScript are object oriented languages, and therefore store and refer to things in a similar manner.

In [ ]:
from pymongo import MongoClient

client = MongoClient()
db = client['uk_police']
collection = db['all_crime']

r = requests.get('https://data.police.uk/api/crimes-no-location?category=all-crime&force=warwickshire&date=2013-09')
r.json()

In [ ]:
collection.insert_many(r.json())

In [ ]:
import pprint as pp
for item in collection.find({ 'month' : '2013-09' }):
    pp.pprint(item)